# データの変形

### ピボットとアンピボット

### ピボット

In [1]:
import pandas as pd

tips = pd.read_csv(
    "https://raw.githubusercontent.com/plotly/datasets/master/tips.csv",
    dtype={
        "sex": "category",
        "smoker": "category",
        "day": "category",
        "time": "category",
    },
)
tips.head()

total_bill   tip     sex smoker  day    time  size
0       16.99  1.01  Female     No  Sun  Dinner     2
1       10.34  1.66    Male     No  Sun  Dinner     3
2       21.01  3.50    Male     No  Sun  Dinner     3
3       23.68  3.31    Male     No  Sun  Dinner     2
4       24.59  3.61  Female     No  Sun  Dinner     4

In [2]:
tips.pivot_table(
    index="smoker",
    columns="time",
    values="total_bill",
)

time       Dinner      Lunch
smoker                      
No      20.095660  17.050889
Yes     21.859429  17.399130

In [3]:
import numpy as np

tips.pivot_table(
    index="smoker",
    columns="time",
    values="total_bill",
    aggfunc="median",
)

time    Dinner  Lunch
smoker               
No      18.265  15.95
Yes     20.695  16.00

In [4]:
tips.pivot_table(
    index=["day", "time"],
    columns=["smoker", "sex"],
    values=["size"],
    aggfunc="count",
)

size                 
smoker          No         Yes     
sex         Female Male Female Male
day  time                          
Fri  Dinner      1    2      4    5
     Lunch       1    0      3    3
Sat  Dinner     13   32     15   27
     Lunch       0    0      0    0
Sun  Dinner     14   43      4   15
     Lunch       0    0      0    0
Thur Dinner      1    0      0    0
     Lunch      24   20      7   10

In [5]:
tips.pivot_table(
    index="time",
    columns="smoker",
    values="size",
    margins=True,
)

smoker        No       Yes       All
time                                
Dinner  2.735849  2.471429  2.630682
Lunch   2.511111  2.217391  2.411765
All     2.668874  2.408602  2.569672

In [6]:
# 下記のコードと同じ処理
# tips.pivot_table(
#     index="day", columns="time", values="tip", aggfunc="count"
# )
pd.crosstab(
    index=tips.loc[:, "day"],
    columns=tips.loc[:, "time"],
    values=tips.loc[:, "tip"],
    aggfunc="count",
)

time  Dinner  Lunch
day                
Fri       12      7
Sat       87      0
Sun       76      0
Thur       1     61

In [7]:
pd.crosstab(
    index=tips.loc[:, "day"],
    columns=tips.loc[:, "time"],
    normalize=True,
)

time    Dinner     Lunch
day                     
Fri   0.049180  0.028689
Sat   0.356557  0.000000
Sun   0.311475  0.000000
Thur  0.004098  0.250000

## アンピボット

In [8]:
pivoted_tips = tips.pivot_table(
    index="smoker",
    columns="time",
    values="total_bill",
).reset_index()
pivoted_tips

time smoker     Dinner      Lunch
0        No  20.095660  17.050889
1       Yes  21.859429  17.399130

In [9]:
pivoted_tips.melt(
    id_vars=["smoker"],
    value_vars=["Dinner", "Lunch"],
    var_name=["time"],
    value_name="total_bill",
)

smoker    time  total_bill
0     No  Dinner   20.095660
1    Yes  Dinner   21.859429
2     No   Lunch   17.050889
3    Yes   Lunch   17.399130

## スタックとアンスタック

### スタック

In [10]:
tips_stacked = tips.stack()
tips_stacked.head(14)

0  total_bill     16.99
   tip             1.01
   sex           Female
   smoker            No
   day              Sun
   time          Dinner
   size               2
1  total_bill     10.34
   tip             1.66
   sex             Male
   smoker            No
   day              Sun
   time          Dinner
   size               3
dtype: object

In [11]:
tips_stacked.index[:14]

MultiIndex([(0, 'total_bill'),
            (0,        'tip'),
            (0,        'sex'),
            (0,     'smoker'),
            (0,        'day'),
            (0,       'time'),
            (0,       'size'),
            (1, 'total_bill'),
            (1,        'tip'),
            (1,        'sex'),
            (1,     'smoker'),
            (1,        'day'),
            (1,       'time'),
            (1,       'size')],
           )

### アンスタック

In [12]:
tips_stacked.unstack()

total_bill   tip     sex smoker   day    time size
0        16.99  1.01  Female     No   Sun  Dinner    2
1        10.34  1.66    Male     No   Sun  Dinner    3
2        21.01   3.5    Male     No   Sun  Dinner    3
3        23.68  3.31    Male     No   Sun  Dinner    2
4        24.59  3.61  Female     No   Sun  Dinner    4
..         ...   ...     ...    ...   ...     ...  ...
239      29.03  5.92    Male     No   Sat  Dinner    3
240      27.18   2.0  Female    Yes   Sat  Dinner    2
241      22.67   2.0    Male    Yes   Sat  Dinner    2
242      17.82  1.75    Male     No   Sat  Dinner    2
243      18.78   3.0  Female     No  Thur  Dinner    2

[244 rows x 7 columns]

### MultiIndexのスタック・アンスタック

In [13]:
groupby_time = tips.groupby("time")[  # ①time列でグルーピング
    # ①グループ化したオブジェクトから列を選択
    ["total_bill", "tip"]
].agg(
    ("mean", "median")
)  # ①平均値と中央値を算出
groupby_time.columns.names = (
    "value",
    "agg",
)  # ②インデックスに名前を設定
groupby_time

value  total_bill               tip       
agg          mean  median      mean median
time                                      
Dinner  20.797159  18.390  3.102670   3.00
Lunch   17.168676  15.965  2.728088   2.25

In [14]:
# level=0がvalue、level=1がagg
groupby_time.columns.names

FrozenList(['value', 'agg'])

In [15]:
groupby_time.stack(0)  # インデックスの順序

agg                     mean  median
time   value                        
Dinner tip          3.102670   3.000
       total_bill  20.797159  18.390
Lunch  tip          2.728088   2.250
       total_bill  17.168676  15.965

In [16]:
groupby_time.stack("agg")  # インデックスの名前

value          total_bill       tip
time   agg                         
Dinner mean     20.797159  3.102670
       median   18.390000  3.000000
Lunch  mean     17.168676  2.728088
       median   15.965000  2.250000

In [17]:
groupby_time_stack = groupby_time.stack("value")
groupby_time_stack

agg                     mean  median
time   value                        
Dinner tip          3.102670   3.000
       total_bill  20.797159  18.390
Lunch  tip          2.728088   2.250
       total_bill  17.168676  15.965

In [18]:
# level=0がtime、level=1がvalue
groupby_time_stack.index.names

FrozenList(['time', 'value'])

In [19]:
groupby_time_stack.unstack(1)

agg         mean            median           
value        tip total_bill    tip total_bill
time                                         
Dinner  3.102670  20.797159   3.00     18.390
Lunch   2.728088  17.168676   2.25     15.965

In [20]:
groupby_time_stack.unstack("time")

agg              mean            median        
time           Dinner      Lunch Dinner   Lunch
value                                          
tip          3.102670   2.728088   3.00   2.250
total_bill  20.797159  17.168676  18.39  15.965

## ダミー変数

In [21]:
pd.get_dummies(tips.loc[:, "day"])

Fri    Sat    Sun   Thur
0    False  False   True  False
1    False  False   True  False
2    False  False   True  False
3    False  False   True  False
4    False  False   True  False
..     ...    ...    ...    ...
239  False   True  False  False
240  False   True  False  False
241  False   True  False  False
242  False   True  False  False
243  False  False  False   True

[244 rows x 4 columns]

In [22]:
pd.get_dummies(tips, columns=["smoker", "time"]).head()

total_bill   tip     sex  day  size  smoker_No  smoker_Yes  time_Dinner  \
0       16.99  1.01  Female  Sun     2       True       False         True   
1       10.34  1.66    Male  Sun     3       True       False         True   
2       21.01  3.50    Male  Sun     3       True       False         True   
3       23.68  3.31    Male  Sun     2       True       False         True   
4       24.59  3.61  Female  Sun     4       True       False         True   

   time_Lunch  
0       False  
1       False  
2       False  
3       False  
4       False

## 要素の展開

In [23]:
students = pd.DataFrame(
    {
        "名前": ["寺田", "辻"],
        "履修科目": [
            ["国語", "英語", "数学"],
            ["英語", "物理"],
        ],
        "得点": [[78, 65, 89], [90, 82]],
    }
)
students

名前          履修科目            得点
0  寺田  [国語, 英語, 数学]  [78, 65, 89]
1   辻      [英語, 物理]      [90, 82]

In [24]:
students.loc[:, "得点"].explode()

0    78
0    65
0    89
1    90
1    82
Name: 得点, dtype: object

In [25]:
students.explode(["履修科目", "得点"])

名前 履修科目  得点
0  寺田   国語  78
0  寺田   英語  65
0  寺田   数学  89
1   辻   英語  90
1   辻   物理  82